In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict

import sys
sys.path.append('..')
from model_utils import prompt_classification

cols_output = ['final_answer_found',
               'final_model_queries', 'final_num_suffixes_checked']

In [7]:
import numpy as np
class fake_args:
    template_num_task_phrasing = 0
    max_dset_size = 200
    max_digit = 10
    seed = 1
    train_split_frac = 0.9 # we'll just test on 10% so this runs more efficiently
    task_name = 'sst2_train'

    # these will be varied
    n_shots = 1
args = fake_args()
np.random.seed(args.seed)


In [4]:
checkpoint = 'EleutherAI/gpt-j-6B'
task_name = 'sst2_train'
d = defaultdict(list)

print('loading', checkpoint)
model = prompt_classification.create_model(checkpoint)

loading EleutherAI/gpt-j-6B


In [8]:
import data
data.get_data(
                    args, args.task_name, n_shots=args.n_shots,
                    train_split_frac=args.train_split_frac
                )

Using custom data configuration default
Found cached dataset sst2 (/home/johnmorris/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
Loading cached processed dataset at /home/johnmorris/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-5775b21d75a9c311.arrow


((Dataset({
      features: ['idx', 'sentence', 'label', 'input', 'output', 'text', '__index_level_0__'],
      num_rows: 180
  }),
  Dataset({
      features: ['idx', 'sentence', 'label', 'input', 'output', 'text', '__index_level_0__'],
      num_rows: 20
  })),
 <function data.get_data.<locals>.check_answer_func(x)>,
 'Answer Yes if the input is positive and No if the input is negative. ')

In [9]:
from model_utils import prompt_classification
import data

print('calculating accs...')
for n_shots in [1, 6]: 
    for restrict_to_valid_answers in [True, False]:
        for prompt in ['', 'manual']:
                args.n_shots = n_shots
                (dset, dset_test), check_answer_func, descr = data.get_data(
                    args, args.task_name, n_shots=args.n_shots,
                    train_split_frac=args.train_split_frac
                )
                d['checkpoint'].append(checkpoint)
                d['prompt'].append(prompt)
                d['task_name'].append(task_name)
                d['n_shots'].append(n_shots)
                d['restrict_to_valid_answers'].append(restrict_to_valid_answers)
                if prompt == 'manual':
                    prompt_actual = descr
                else:
                    prompt_actual = prompt
                d['prompt_actual'].append(prompt_actual)
                batch_size = 4
                loss, acc = prompt_classification.test_model_on_task_with_prefix(
                    dset=dset, model=model,
                    prefix=prompt_actual,
                    multi_token=False,
                    verbose=False,
                    restrict_to_valid_answers=restrict_to_valid_answers,
                    batch_size=batch_size,
                )
                d['acc'].append(acc)

calculating accs...


Using custom data configuration default
Found cached dataset sst2 (/home/johnmorris/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
Loading cached processed dataset at /home/johnmorris/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-5775b21d75a9c311.arrow
Using custom data configuration default
Found cached dataset sst2 (/home/johnmorris/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
Loading cached processed dataset at /home/johnmorris/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-5775b21d75a9c311.arrow
Using custom data configuration default
Found cached dataset sst2 (/home/johnmorris/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
Loading cached proces

In [10]:
d

defaultdict(list,
            {'checkpoint': ['EleutherAI/gpt-j-6B',
              'EleutherAI/gpt-j-6B',
              'EleutherAI/gpt-j-6B',
              'EleutherAI/gpt-j-6B',
              'EleutherAI/gpt-j-6B',
              'EleutherAI/gpt-j-6B',
              'EleutherAI/gpt-j-6B',
              'EleutherAI/gpt-j-6B'],
             'prompt': ['',
              'manual',
              '',
              'manual',
              '',
              'manual',
              '',
              'manual'],
             'task_name': ['sst2_train',
              'sst2_train',
              'sst2_train',
              'sst2_train',
              'sst2_train',
              'sst2_train',
              'sst2_train',
              'sst2_train'],
             'n_shots': [1, 1, 1, 1, 6, 6, 6, 6],
             'restrict_to_valid_answers': [True,
              True,
              False,
              False,
              True,
              True,
              False,
              False],
          

In [11]:
import pandas as pd
df = pd.DataFrame.from_dict(d)
df

,checkpoint,prompt,task_name,n_shots,restrict_to_valid_answers,prompt_actual,acc
0,EleutherAI/gpt-j-6B,,sst2_train,1,True,,56.111111
1,EleutherAI/gpt-j-6B,manual,sst2_train,1,True,Answer Yes if the input is positive and No if ...,62.777778
2,EleutherAI/gpt-j-6B,,sst2_train,1,False,,0.000000
3,EleutherAI/gpt-j-6B,manual,sst2_train,1,False,Answer Yes if the input is positive and No if ...,58.888889
4,EleutherAI/gpt-j-6B,,sst2_train,6,True,,75.555556
5,EleutherAI/gpt-j-6B,manual,sst2_train,6,True,Answer Yes if the input is positive and No if ...,77.777778
6,EleutherAI/gpt-j-6B,,sst2_train,6,False,,75.555556
7,EleutherAI/gpt-j-6B,manual,sst2_train,6,False,Answer Yes if the input is positive and No if ...,77.777778


In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(d)
df